<a href="https://colab.research.google.com/github/ana-isabellagf/earth_engine_python/blob/main/extract_lulc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Série temporal do Uso e Cobertura do Solo (LULC)**
## **Extração e visualização**

Fiz um artigo no Medium falando sobre esse código, aqui está: [Série temporal LULC](https://medium.com/@aisabellaguimaraesf/série-temporal-do-uso-e-cobertura-do-solo-lulc-extração-e-visualização-52c59c5e0998)

Autor: Ana Isabella Guimarães Ferreira (aisabellaguimaraesf@gmail.com)

LinkedIn: www.linkedin.com/in/ana-isabella-g-ferreira


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q rasterio

In [ ]:
# Requirements
import time
import os
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio import features
from rasterio.mask import mask
import geemap
from shapely.geometry import shape
import folium
from folium.plugins import TimestampedGeoJson
from shapely.geometry import mapping
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='my-project')

In [ ]:
# CRS CODE - Brazil Data Cube
# https://brazil-data-cube.github.io/products/specifications/bdc-grid.html
crs_code = '+proj=aea +lat_0=-12 +lon_0=-54 +lat_1=-2 +lat_2=-22 +x_0=5000000 +y_0=10000000 +ellps=GRS80 +units=m +no_defs'

In [ ]:
def extract_lulc_images(start_year, end_year, geom, aoi, crs_code):
    """
    Extrai imagens LULC do Earth Engine para uma área de interesse e retorna um dicionário de GeoDataFrames por ano.

    Args:
        start_year (int): Ano inicial.
        end_year (int): Ano final.
        geom (ee.Geometry): Geometria da área de interesse.
        aoi (gpd.GeoDataFrame): GeoDataFrame com os limites da área de interesse.
        crs_code (str): Código CRS para os GeoDataFrames resultantes.

    Returns:
        dict: Dicionário contendo GeoDataFrames de LULC por ano.
    """
    lulc_by_year = {}

    # Obter o tempo inicial
    start_time = time.time()

    for ano in range(start_year, end_year + 1):
        # Acessar o asset do Mapbiomas
        dataset = ee.Image('projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1').select(f'classification_{ano}').clip(geom)
        output_filename = f'temp_mapbiomas_{ano}.tif'
        geemap.ee_export_image(dataset, output_filename, scale=30, crs="EPSG:4326", region=geom)

        # Abrir a imagem localmente usando rasterio
        with rasterio.open(output_filename) as src:
            # Cortando o raster usando os limites do GeoDataFrame 'area'
            out_image, out_transform = mask(src, aoi.geometry, crop=True)
            out_meta = src.meta.copy()

        # Criar um rasterio feature object com o mapbiomas
        shapes = features.shapes(out_image[0], transform=out_transform)

        # Listas vazias para armazenar classes de LULC e geometrias
        value_class = []
        geometry = []

        # Preencher as listas com as geometrias e valores das classes
        for shapedict, value in shapes:
            value_class.append(value)
            geometry.append(shape(shapedict))

        # Criar um GeoDataframe com os valores das classes e geometria
        lulc_gdf = gpd.GeoDataFrame({f'lulc_{ano}': value_class, 'geometry': geometry}, crs=crs_code)

        # Armazenar o GeoDataFrame no dicionário
        lulc_by_year[ano] = lulc_gdf

        # Remover o arquivo TIFF temporário
        os.remove(output_filename)

    # Obter o tempo final
    end_time = time.time()

    # Calcular e exibir o tempo de execução
    execution_time = end_time - start_time
    print(f"Tempo de execução: {execution_time} segundos")

    return lulc_by_year

# Área de interesse
aoi = gpd.read_file('aoi_lulc_dynamic.zip')

# Geometrias para extração dos dados
bounds = aoi.geometry.total_bounds
geom = ee.Geometry.Rectangle(bounds.tolist())

# Aplicando a função
lulc_by_year = extract_lulc_images(1985, 2022, geom, aoi, crs_code)

In [ ]:
# Extrair latitude e longitude
centroide = aoi.centroid.geometry.iloc[0]
lon = centroide.x
lat = centroide.y

# Legenda Mapbiomas
# https://brasil.mapbiomas.org/wp-content/uploads/sites/4/2023/09/Codigos-da-legenda-colecao-8-csv.zip
codigos = pd.read_csv('Codigos-da-legenda-colecao-8.csv', sep=';')
codigos.head()

# Criar dicionário com as cores correspondentes às classes
class_color_map = dict(zip(codigos['Class_ID'], codigos['Color']))

In [ ]:
# Função para converter GeoDataFrame em GeoJSON
def gdf_to_geojson(gdf, properties, date, class_color_map):
    features = []
    for _, row in gdf.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': mapping(row.geometry),
            'properties': {prop: row[prop] for prop in properties}
        }
        # Adicionar propriedade de cor com base na classe LULC
        lulc_value = row[properties[0]]
        feature['properties']['color'] = class_color_map.get(lulc_value, "#000000")  # Preto como fallback
        # Adicionar a propriedade de tempo
        feature['properties']['time'] = date
        # Adicionar o nome da classe LULC
        label = codigos.loc[codigos["Class_ID"] == lulc_value, "Description"].values
        if len(label) > 0:
            feature['properties']['class_name'] = label[0]
        else:
            feature['properties']['class_name'] = f'Class {lulc_value}'  # Rótulo padrão se a classe não existir em 'codigos'
        features.append(feature)
    return {'type': 'FeatureCollection', 'features': features}

# Função de estilo para o GeoJSON
def style_function(feature):
    return {
        'fillColor': feature['properties']['color'],
        'color': None,
        'weight': 0,
        'fillOpacity': 1,
    }

In [ ]:
# Lista para armazenar os GeoJSONs
geojson_list = []
unique_classes = set()  # Conjunto para armazenar as classes únicas

# Iterar pelos anos e converter os GeoDataFrames para GeoJSON
for ano_plot in range(2012, 2023):
    gdf = lulc_by_year[ano_plot]
    # Converter valores float para inteiros na coluna lulc_ano
    gdf[f'lulc_{ano_plot}'] = gdf[f'lulc_{ano_plot}'].astype(int)
    # Adicionar classes únicas ao conjunto
    unique_classes.update(gdf[f'lulc_{ano_plot}'].unique())
    # Converter GeoDataFrame para GeoJSON
    geojson = gdf_to_geojson(gdf, properties=[f'lulc_{ano_plot}'], date=f'{ano_plot}-01-01', class_color_map=class_color_map)
    geojson_list.append(geojson)

# Criar uma lista de GeoJSONs com estilo aplicado
styled_features = []
for geojson in geojson_list:
    for feature in geojson['features']:
        feature['properties']['style'] = style_function(feature)
        styled_features.append(feature)

In [ ]:
# Defina as coordenadas centrais da sua área de interesse
centro_mapa = (lat, lon)
zoom_inicial = 12.5

# Criação do mapa base
m = folium.Map(location=centro_mapa, zoom_start=zoom_inicial)

# Criar GeoJsonTooltip
tooltip = folium.GeoJsonTooltip(
    fields=['class_name'],
    aliases=['Class: '],
    localize=True
)

# Adicionar camada temporal ao mapa
timestamped_geojson = TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': styled_features
    },
    transition_time=200,  # tempo de transição entre os anos
    period='P1Y',  # período de 1 ano
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY',
    time_slider_drag_update=True
)

# Adicionar camada temporal ao mapa
timestamped_geojson.add_to(m)

# Adicionar GeoJson com tooltips ao mapa
folium.GeoJson(
    timestamped_geojson.data,
    style_function=style_function,
    tooltip=tooltip
).add_to(m)

# Salvar o mapa como um arquivo HTML
m.save('historico_LULC.html')